This Query is created by user undefined

In [2]:
%% sql
WITH AGG_POLICY_DATA AS (
    SELECT 
        AGG_POLICY_MASTER_REPORT_DAILY.POLICY_NUMBER,
        AGG_POLICY_ENDORSMENT_DAILY.POLICY_STATUS,
        AGG_POLICY_ENDORSMENT_DAILY.Endorsement_Effective_from_date AS DateOfPolicyOpening,
        AGG_POLICY_ENDORSMENT_DAILY.Endorsement_Effective_To_date AS DateOfPolicyClosing,
        AGG_POLICY_MASTER_REPORT_DAILY.OFFICE_CODE AS BranchCodeOfPolicy,
        AGG_POLICY_ENDORSMENT_DAILY.Endorsement_Number,
        AGG_POLICY_MASTER_REPORT_DAILY.PROPOSER_CODE AS CustomerID,
        AGG_POLICY_MASTER_REPORT_DAILY.intermediary_code AS AgentID,
        AGG_POLICY_MASTER_REPORT_DAILY.SP_CODE AS BrokerID,
        AGG_POLICY_MASTER_REPORT_DAILY.fulfiller_code AS FulfillerID,
        AGG_POLICY_ENDORSMENT_DAILY.SOURCE AS Source,
        GETDATE() AS DateOfExtraction,
        CASE 
            WHEN AGG_POLICY_ENDORSMENT_DAILY.SOURCE = 'PREMIA' THEN (
                SELECT DISTINCT POLH_SYS_ID 
                FROM PGITH_POLICY 
                WHERE POLH_NO = AGG_POLICY_ENDORSMENT_DAILY.POLICY_NUMBER
            )
            ELSE (
                SELECT POLICY_ID 
                FROM T_RPDB_POLICY_MASTER 
                WHERE POLICY_NUMBER = AGG_POLICY_ENDORSMENT_DAILY.POLICY_NUMBER
            )
        END AS SystemIDForPolicy,
        AGG_POLICY_ENDORSMENT_DAILY.Policy_Endorsement_no_idx AS EndorsementSequenceNumber,
        AGG_POLICY_ENDORSMENT_DAILY.TRANSACTION_TYPE AS PolicyType
    FROM AGG_POLICY_ENDORSMENT_DAILY
    INNER JOIN AGG_POLICY_MASTER_REPORT_DAILY 
    ON AGG_POLICY_MASTER_REPORT_DAILY.POLICY_NUMBER = AGG_POLICY_ENDORSMENT_DAILY.POLICY_NUMBER
)
SELECT 
    AGG_POLICY_DATA.POLICY_NUMBER AS PolicyNumber,
    AGG_POLICY_DATA.POLICY_STATUS AS PolicyStatus,
    AGG_POLICY_DATA.DateOfPolicyOpening,
    AGG_POLICY_DATA.DateOfPolicyClosing,
    AGG_POLICY_DATA.BranchCodeOfPolicy,
    AGG_POLICY_DATA.Endorsement_Number AS EndorsementNumber,
    AGG_POLICY_DATA.CustomerID AS CustomerID,
    AGG_POLICY_DATA.AgentID AS AgentID,
    AGG_POLICY_DATA.BrokerID AS BrokerID,
    AGG_POLICY_DATA.FulfillerID AS FulfillerID,
    AGG_POLICY_DATA.Source,
    AGG_POLICY_DATA.DateOfExtraction,
    AGG_POLICY_DATA.SystemIDForPolicy,
    AGG_POLICY_DATA.EndorsementSequenceNumber,
    AGG_POLICY_DATA.PolicyType
FROM AGG_POLICY_DATA